In [2]:
!pip install natasha

   ---------------------------------------- 0.0/34.4 MB ? eta -:--:--
   ---------------------------------------- 0.0/34.4 MB ? eta -:--:--
   ---------------------------------------- 0.2/34.4 MB 2.9 MB/s eta 0:00:12
    --------------------------------------- 0.5/34.4 MB 4.3 MB/s eta 0:00:08
   - -------------------------------------- 1.1/34.4 MB 6.4 MB/s eta 0:00:06
   - -------------------------------------- 1.7/34.4 MB 7.8 MB/s eta 0:00:05
   -- ------------------------------------- 2.5/34.4 MB 9.5 MB/s eta 0:00:04
   ---- ----------------------------------- 3.5/34.4 MB 11.2 MB/s eta 0:00:03
   ----- ---------------------------------- 4.4/34.4 MB 12.2 MB/s eta 0:00:03
   ------ --------------------------------- 5.5/34.4 MB 13.5 MB/s eta 0:00:03
   ------- -------------------------------- 6.7/34.4 MB 14.8 MB/s eta 0:00:02
   -------- ------------------------------- 7.6/34.4 MB 15.3 MB/s eta 0:00:02
   ---------- ----------------------------- 8.8/34.4 MB 16.1 MB/s eta 0:00:02
   ----

In [21]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder, FunctionTransformer
from sklearn.pipeline import Pipeline
from natasha import Segmenter, NewsEmbedding, NewsMorphTagger, MorphVocab, Doc
import dill

In [4]:
# Загрузим и сразу подготовим тестовые данные

train = pd.read_csv('data/train.csv')
test_data = pd.read_csv('data/test.csv')
sample = pd.read_csv('data/sample_submission.csv')

test = pd.merge(test_data, sample, on='oid', how='inner')

In [5]:
# Закодируем целевую переменную и удалим лишние столбцы

le = LabelEncoder()

train['labels'] = le.fit_transform(train['category'])
test['labels'] = le.transform(test['category'])

train = train.drop(columns=['oid', 'category'])
test = test.drop(columns=['oid', 'category'])

print(train.shape)
print(train.head(3))
print()
print(test.shape)
print(test.head(3))

(38740, 2)
                                                text  labels
0  Волшебные фото Виктория Поплавская ЕвгенияМедв...      12
1  Возвращение в подземелье Треша 33 Эйфория тупо...       5
2  Лучшие чешские вратари – Доминик Доминатор Гаш...       6

(26260, 2)
                                                text  labels
0  СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...       4
1  Естественное восстановление после тяжелой трен...       8
2  Тема нарядов продолжается Одна из британских ж...      10


In [6]:
# Избавимся от дубликатов

train.drop_duplicates(inplace=True)
test.drop_duplicates(inplace=True)

print(train.shape)
print(test.shape)

(35965, 2)
(24936, 2)


In [47]:
# Инициализируем инструменты, создадим функцию для обработки текста

segmenter = Segmenter()
emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
morph_vocab = MorphVocab()

def lemmatize(text):
    doc = Doc(text)
    doc.segment(segmenter)
    doc.tag_morph(morph_tagger)
    for token in doc.tokens:
        token.lemmatize(morph_vocab)
    lemmas = [token.lemma.lower() for token in doc.tokens if token.lemma.isalpha()]
    return  ' '.join(lemmas)

In [8]:
# Обработаем текст

train['lemm'] = train['text'].apply(lemmatize)
train.head()

,text,labels,lemm
0,Волшебные фото Виктория Поплавская ЕвгенияМедв...,12,волшебный фото виктория поплавский евгениямедв...
1,Возвращение в подземелье Треша 33 Эйфория тупо...,5,возвращение в подземелье треш эйфория тупость ...
2,Лучшие чешские вратари – Доминик Доминатор Гаш...,6,хороший чешский вратарь доминик доминатор гаше...
3,Rtokenoid Warhammer40k валрак решил нас подкор...,3,rtokenoid warhammer k валрак решить мы подкорм...
4,Шестеркин затаскивает Рейнджерс в финал Восточ...,7,шестеркин затаскивает рейнджерс в финал восточ...


In [9]:
test['lemm'] = test['text'].apply(lemmatize)
test.head()

,text,labels,lemm
0,СПОЧНО СООБЩЕСТВО ПРОДАЕТСЯ ЗА 1300Р ЗА ПОКУПК...,4,спочно сообщество продаваться за р за покупка ...
1,Естественное восстановление после тяжелой трен...,8,естественный восстановление после тяжелый трен...
2,Тема нарядов продолжается Одна из британских ж...,10,тема наряд продолжаться один из британский жур...
3,Привет Избранный. Ты спрашиваешь себя ЧТО здес...,4,привет избранный ты спрашивать себя что здесь ...
4,КОРОЛЬ ПЯТИСОТНИКОВ В ДЕЛЕ Андрей Рублев успеш...,10,король пятисотник в дело андрей рублев успешно...


In [11]:
# Разделим данные для обучением, векторизуем текст

vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(train['lemm'])
y_train = train['labels']
X_test = vectorizer.transform(test['lemm'])
y_test = test['labels']

In [12]:
# Обучим модели

lr = LogisticRegression().fit(X_train, y_train)

rf = RandomForestClassifier().fit(X_train, y_train)

In [13]:
# Проверим метрики

print("Logistic Regression:\n", classification_report(y_test, lr.predict(X_test), target_names=le.classes_))
print("Random Forest:\n", classification_report(y_test, rf.predict(X_test), target_names=le.classes_))

Logistic Regression:
               precision    recall  f1-score   support

   athletics       0.94      0.78      0.85      2085
   autosport       0.91      0.82      0.86      1907
  basketball       0.89      0.84      0.87      1711
  boardgames       0.93      0.92      0.92      1795
      esport       0.75      0.85      0.79      1855
     extreme       0.64      0.76      0.69      1866
    football       0.76      0.83      0.79      1805
      hockey       0.88      0.81      0.84      2120
martial_arts       0.72      0.88      0.79      1650
   motosport       0.92      0.91      0.91      1850
      tennis       0.97      0.93      0.95      2002
  volleyball       0.91      0.81      0.86      2129
winter_sport       0.87      0.87      0.87      2161

    accuracy                           0.85     24936
   macro avg       0.85      0.85      0.85     24936
weighted avg       0.86      0.85      0.85     24936

Random Forest:
               precision    recall  f1-sco

In [48]:
pipeline = Pipeline([
    ('vector', vectorizer),
    ('model', lr)
])

In [49]:
with open('LogRegModel.dill', 'wb') as file:
    dill.dump(pipeline, file)

In [50]:
# Проверим работу локально

text_test = 'Отличный прыжок нашего спортсмена со снаряда, идеальное приземление'
text_test2 = 'Медведев закончил игру подачей навылет. Гейм, сет, матч'

def predict(text):
    text = lemmatize(text)
    predict = pipeline.predict([text])
    return le.inverse_transform(predict)[0]

In [51]:
# Проверим ответ модели

print(predict(text_test))
print()
print(predict(text_test2))

volleyball

tennis
